### Prob 1. STDescManager::extract_corner

点投影到平面

```cpp
Eigen::Vector3d cur_project;
cur_project[0] = (-A * (B * y + C * z + D) + x * (B * B + C * C)) /
                    (A * A + B * B + C * C);
cur_project[1] = (-B * (A * x + C * z + D) + y * (A * A + C * C)) /
                    (A * A + B * B + C * C);
cur_project[2] = (-C * (A * x + B * y + D) + z * (A * A + B * B)) /
                    (A * A + B * B + C * C);
```

In [1]:
from sympy import *

In [22]:
A, B, C, D = symbols('A B C D')
cx, cy, cz = symbols('cx cy cz')
x, y, z = symbols('x y z')
t = symbols('t')

In [35]:
plane = Matrix([A, B, C])
plane_norm = Matrix([A, B, C]) / Matrix([A, B, C]).norm()
plane_center = Matrix([cx, cy, cz])
point = Matrix([x, y, z])

In [25]:
proj_point = t * plane_norm + point

In [36]:
res = solve([plane.dot(proj_point) + D], [t])

In [37]:
pp = proj_point.subs(t, res[t])

In [38]:
simplify(pp[0])

(-A*B*y - A*C*z - A*D + B**2*x + C**2*x)/(A**2 + B**2 + C**2)

In [39]:
simplify(pp[1])

(A**2*y - A*B*x - B*C*z - B*D + C**2*y)/(A**2 + B**2 + C**2)

In [40]:
simplify(pp[2])

(A**2*z - A*C*x + B**2*z - B*C*y - C*D)/(A**2 + B**2 + C**2)

### Prob 2. STDescManager::corner_extractor

近似法向量判定

```cpp
Eigen::Vector3d normal_diff = projection_normal - normal;
Eigen::Vector3d normal_add = projection_normal + normal;
if (normal_diff.norm() < 0.5 || normal_add.norm() < 0.5) {
    skip_flag = true;
}

```

In [8]:
theta, d = symbols("theta, d", real=True)

In [9]:
x_norm = Matrix([1.0, 0.0, 0.0])
near_norm = Matrix([cos(theta), sin(theta), 0.0])

In [10]:
res = solve([(x_norm - near_norm).norm() - d], [theta])

In [19]:
res[0][0].subs(d, 0.5) / 3.1415926 * 180.0

-28.9550248657793

In [21]:
res[1][0].subs(d, 0.5) / 3.1415926 * 180.0

28.9550248657793

## Prob 3. STDescManager::candidate_selector

**可搜索空间**

分析 thr 对可搜索空间的大小有怎样的影响

下面的代码需要放在单独的脚本运行，因为生成的图象是 3d 图像，jupyter 中画出来的不可交互

```cpp
double thr = 1.5;
position.x = (int)(src_std.side_length_[0] + voxel_inc[0]);
position.y = (int)(src_std.side_length_[1] + voxel_inc[1]);
position.z = (int)(src_std.side_length_[2] + voxel_inc[2]);
Eigen::Vector3d voxel_center(
    (double)position.x + 0.5,
    (double)position.y + 0.5,
    (double)position.z + 0.5
);
if ((src_std.side_length_ - voxel_center).norm() < thr) {}

```

In [22]:
import matplotlib.pyplot as plt
import numpy as np
import itertools

In [44]:
side_lengths = np.array(list(itertools.product([0.25, 0.75], repeat=3)))

In [60]:
thr = 1.5
voxel_centerss = []

for side_length in side_lengths:
    voxel_centers = [side_length]
    for search_dir in itertools.product([-1, 0, 1], repeat=3):
        position = side_length.astype(int) + np.asarray(search_dir)
        voxel_center = position.astype(float) + 0.5
        if np.linalg.norm(side_length - voxel_center) < thr:
            voxel_centers.append(voxel_center)
    voxel_centerss.append(voxel_centers)
            

In [61]:
fig, axs = plt.subplots(2, 4, figsize=(10, 10), subplot_kw={'projection': '3d'})
for ax, voxel_centers in zip(axs.flatten(), voxel_centerss):
    ax.set_xlim(-1, 2)
    ax.set_ylim(-1, 2)
    ax.set_zlim(-1, 2)
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.set_title(f'{voxel_centers[0]}')
    ax.scatter(*np.array(voxel_centers[1:]).T, c='blue', alpha=0.5)
    ax.scatter(*np.array(voxel_centers[0]).T, c='green', alpha=1)
plt.show()

[array([-0.75, -0.75, -0.75])]


## Prob 4. STDescManager::triangle_solver

分析 ICP 的数学原理

```cpp
Eigen::Matrix3d src = Eigen::Matrix3d::Zero();
Eigen::Matrix3d ref = Eigen::Matrix3d::Zero();
src.col(0) = std_pair.first.vertex_A_ - std_pair.first.center_;
src.col(1) = std_pair.first.vertex_B_ - std_pair.first.center_;
src.col(2) = std_pair.first.vertex_C_ - std_pair.first.center_;
ref.col(0) = std_pair.second.vertex_A_ - std_pair.second.center_;
ref.col(1) = std_pair.second.vertex_B_ - std_pair.second.center_;
ref.col(2) = std_pair.second.vertex_C_ - std_pair.second.center_;
Eigen::Matrix3d covariance = src * ref.transpose();
Eigen::JacobiSVD<Eigen::MatrixXd> svd(covariance, Eigen::ComputeThinU |
                                                    Eigen::ComputeThinV);
Eigen::Matrix3d V = svd.matrixV();
Eigen::Matrix3d U = svd.matrixU();
rot = V * U.transpose();
if (rot.determinant() < 0) {
Eigen::Matrix3d K;
K << 1, 0, 0, 0, 1, 0, 0, 0, -1;
rot = V * K * U.transpose();
}
t = -rot * std_pair.first.center_ + std_pair.second.center_;
```

In [1]:
import sympy as sp

#### 问题描述

在 $d$ 维空间中，存在两个点集 $P=\{p_1, ..., p_n\}$ $Q=\{q_1, ..., q_n\}$，其中 $p_i$ 与 $q_i$ 对应

我们希望找到一个旋转矩阵 $R$ 和一个平移向量 $t$ ，满足对齐后的所有点距离的加权平方和最小

$$
(R, t) = \underset{R\in SO(d), t \in \mathbb{R}^d}{\mathrm{argmin}} \sum^{n}_{i=1}\|(Rp_i + t) - q_i\|^2
$$

In [2]:
p = sp.MatrixSymbol('p', 3, 1)
q = sp.MatrixSymbol('q', 3, 1)
R = sp.MatrixSymbol('R', 3, 3)
t = sp.MatrixSymbol('t', 3, 1)
f = sp.Function('f')(t)

## 代价函数求和符号中的其中一项
cost_vec = R @ p + t - q
f = cost_vec.T @ cost_vec

首先处理平移向量 $t$，将代价函数看作是 $t$ 的函数，对其求一阶导，然后令导数为零，解出 t

对代价函数求和符号中的一项求导

In [3]:
f.diff(t)

2*R*p - 2*q + 2*t

如此，我们可以轻松的得到完整的表达式

$$
\frac{df}{dt} = 2nt + 2R\sum^n_i p_i - 2\sum^n_i q_i
$$

将点求和的部分换成 **中心点 * n** 的形式去掉求和符号，得到

$$
\frac{df}{dt} = 2nt + 2nR\bar{p} - 2n\bar{q}
$$

其中 $\bar{p}$，$\bar{q}$ 分别是点云 $P$ 和 $Q$ 的中心点

这样就能得到 t 的表达式

$$
t = \bar{q} - R\bar{p}
$$

将 $t$ 带入上面的残差方程得到



In [23]:
p_bar = sp.MatrixSymbol("\\bar{p}", 3, 1)
q_bar = sp.MatrixSymbol("\\bar{q}", 3, 1)


cost_vec.subs(t, q_bar - R @ p_bar).expand()


-R*\bar{p} + R*p + \bar{q} - q

使用 $x_i$, $y_i$ 替换 $p - \bar{p}$, $q - \bar{q}$ ，我们将得到

$$
R = \underset{R\in SO(d)}{\mathrm{argmin}} \sum^{n}_{i=1}\|Rx_i - y_i\|^2
$$

对求和项进行如下化简

In [28]:
x = sp.MatrixSymbol('x', 3, 1)
y = sp.MatrixSymbol('y', 3, 1)

cost_vec = R @ x - y
f = (cost_vec.T @ cost_vec).expand()
f

x.T*R.T*R*x - x.T*R.T*y - y.T*R*x + y.T*y

In [33]:
# R^TR = I
f = f.subs(R.T @ R, sp.eye(3))
f

x.T*x - y.T*R*x - y.T*R*x + y.T*y

In [34]:
# 这是一个标量 -> x^T R^T y = (y^T R x)^T
f = f.subs(x.T @ R.T @ y, y.T @ R @ x).simplify()
f

x.T*x - y.T*R*x - y.T*R*x + y.T*y

这样目标函数就能简化为

$$
R = \underset{R\in SO(d)}{\mathrm{argmax}} \sum^{n}_{i=1}(y^T_i R x_i)^2
$$

想到矩阵的 **迹** 是对角元素的和，那么我们完全可以将上述方程改造成矩阵乘法求迹的形式，
完全规避求和符号的干扰

$$
R = \underset{R\in SO(d)}{\mathrm{argmax}} \  tr (Y^T R X)
$$

其中

$$
Y^T = \begin{bmatrix}
y^T_1 \\
... \\
y^T_n
\end{bmatrix}
$$

$$
X = \begin{bmatrix}
x_1 & ... & x_n \\
\end{bmatrix}
$$

这样就可以应用迹的性质对上式进行化简 [$tr(AB) = tr(BA)$](https://zh.wikipedia.org/zh-hans/%E8%B7%A1#%E7%9F%A9%E9%98%B5%E4%B9%98%E7%A7%AF%E7%9A%84%E8%BF%B9%E6%95%B0)

$$
R = \underset{R\in SO(d)}{\mathrm{argmax}} \  tr (R X Y^T)
$$

再对 $X Y^T$ 进行分解得到 $X Y^T = U \Sigma V^T$，得到

$$
tr (R X Y^T) = tr(R U \Sigma V^T) = tr(\Sigma V^T R U)
$$

因为 $V$ $R$ $U$ 都是正交矩阵，因此他们的乘积仍然是正交矩阵，而[正交矩阵所有元素都小于1](https://math.stackexchange.com/a/3285427)

因此，为了最大化迹的结果，让 $V^T R U$ 的结果是单位矩阵就是最优结果，这样我们就得到了 R
的表达式

$$
\begin{aligned}
I &= V^T R U \\
R &= V U^T
\end{aligned}
$$

由于反射矩阵和旋转矩阵都是正交矩阵，因此需要检测 $R$ 的行列式是否为整数，若为负数则需要
反转旋转矩阵，让其行列式变为正数，这种情况只有在匹配点处于同一平面上才会出现，恰好是我们
现在要处理的三角形匹配问题，关于这个问题的讨论参见 [此论文](https://www.math.pku.edu.cn/teachers/yaoy/Fall2011/arun.pdf)

简单来说就是，所有点共面时，第三个奇异值为 0，对这个奇异值对应的特征向量随意反转都不会影响
分解的正确性，只会影响行列式的结果